<a href="https://colab.research.google.com/github/DanC777/AMSI-VRS-Project/blob/master/Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Use older version of Keras so that embedding layer works

!pip install keras==2.1.1

In [2]:
# Import all relevent packages and methods


import math
from math import sqrt
import random
from numpy.random import seed
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import special
from sklearn import metrics, preprocessing
from sklearn.preprocessing import  MinMaxScaler, StandardScaler
import tensorflow as tf

import keras
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from keras.layers import Dense, Activation, Flatten, Embedding, Concatenate, Input, Dropout
from keras.models import Sequential, Model

from keras.callbacks import EarlyStopping
from keras.layers.core import Dropout
from keras import regularizers
from sklearn.model_selection import StratifiedKFold


from numpy import array
from numpy import argmax
from keras.utils import to_categorical

from sklearn.datasets import make_regression, make_classification
import statsmodels.formula.api as smf

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Regression


## **Regression - 2 continuous, no cats**

In [0]:
## Function to generate synthetic dataset
def get_data1(observations, sigma):

  x1 = np.random.randint(0,100, size = [observations,1]).astype(float)/10
  x2 = np.random.randint(0,50, size = [observations,1]).astype(float)/10
  
  ## Manually set weights
  w1 = 5
  w2 = 2
  
  # Create error term which adds noise to data
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) 
  
  
  x = np.hstack((x1,x2))
  y = np.dot(x1**2,w1) + np.dot(x2,w2)  + e
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df = pd.concat([df1, df2], axis = 1)
  
  df.columns = ["x1", "x2",  "y"]

  return(df)

## Function to split dataframe into testing and training sets
def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)




In [0]:
## Function to compare normal regression and neural network. 
## Train NN once and run model on random test set 30 times to get average test accuracy

def compare_regression_data1(observations, sigma, test_size, vector, learn, epochs, rounds):
  
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  df = get_data1(observations = observations, sigma = sigma)
  x = np.array(df.iloc[:, :-1]) 
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
  
  ## Normalise data
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train)
  
  X_train = scalarX.transform(X_train)
  
  X_test = scalarX.transform(X_test)
  
  
 
  
  def build__regression_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],  activation = 'relu', ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation= 'relu'))
         
        
      model.add(Dense(1, activation = "linear")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 

      model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                optimizer=optimizer,
                metrics=['mse'])
      return model
    
    
    
    
  model = build__regression_model(vector = vector, learn = learn)
  
  # Store training stats
  model.fit(X_train,y_train, epochs=epochs,
                    validation_split=0.2, verbose=0)
  
  
  [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
  y_pred_nn = model.predict(X_test).astype(np.float64)
  
  ## Take RMSE as final metric
  train_nn = sqrt(mse)
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
  
  train_nn_list.append(train_nn)
  test_nn_list.append(test_nn)
  
  
  
  ## Test on multiple test sets
  for i in range(rounds):
    ## OLS
    df = get_data1(observations = observations, sigma = sigma)
    train, test = split(df, test_size = test_size)
    
    results = smf.ols(formula = 'y ~ np.power(x1,2) + x2 ', data = sm.add_constant(train)).fit()
    
    train_pred_ols = results.fittedvalues
    train_ols = sqrt(metrics.mean_squared_error(train_pred_ols, train["y"]))
    
    test_pred_ols = results.predict(test)
    test_ols = sqrt(metrics.mean_squared_error(test_pred_ols, test["y"]))
    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
  
    ## NN

    df = get_data1(observations = observations, sigma = sigma)
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
    
    ## Normalise data
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train)
    
    X_train = scalarX.transform(X_train)
    
    X_test = scalarX.transform(X_test)
    
    [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
    y_pred_nn = model.predict(X_test).astype(np.float64)
    
    ## Take RMSE as final metric
    train_nn = sqrt(mse)
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
    
    train_nn_list.append(train_nn)
    test_nn_list.append(test_nn)
  
  ## Find average MSE and standard deviation of error
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  test_error_ols = np.std(test_ols_list)
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)  
  test_error_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols, test_error_ols, average_train_nn, average_test_nn, test_error_nn)

In [5]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, test_nn, std_nn = compare_regression_data1(observations = 1000, sigma = 0.01, test_size = 0.33, vector = [1024,64,1], learn = 0.01,epochs = 1000, rounds = 30)

print("###########################  RMSE #######################")
print(" Training Error OLS: {}        |  Training Error NN: {} ".format(train_ols, train_nn))
print(" Testing Error OLS: {}         |  Testing Error NN: {}  ".format(test_ols, test_nn))
    
print("######################## Standard Deviation #############")
print(" Testing STD ols: {}           |  Testing std NN: {}   ".format(std_ols, std_nn))




Instructions for updating:
keep_dims is deprecated, use keepdims instead
###########################  RMSE #######################
 Training Error OLS: 0.009964747895744001        |  Training Error NN: 2.507688411155622 
 Testing Error OLS: 0.009991374603937188         |  Testing Error NN: 2.5139748770951673  
######################## Standard Deviation #############
 Testing STD ols: 0.0003311150222639846           |  Testing std NN: 0.11838283789553718   


## Regression - 2 continuous, one cat, linear

In [0]:
def get_data2(observations, sigma, representation):
 

  x1 = np.random.randint(0,100, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m feature
   
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  ## Add categorical variable (ranging between 1,7 to represent days of week)
  
  
  ## Convert to dataframe to use "get_dummies" function
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  
  a = np.array([[2,5,1,1,1,1,7,7]]) ## Set cooeficients manually
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  y = np.dot(x,a.T) + e
  
  ## Create dataframe's with one-hot encoding for OLS and without for NN
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  
  
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)


In [0]:
def compare_regression_data2(observations, sigma, test_size, vector, learn, output, epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  #NN
  df = get_data2(observations = observations, sigma = sigma, representation = "nn"  )
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
        
  ## Normalise  
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  # Define the embedding input
  cat_input = Input(shape=(1,)) 
  
  ## Numerical inut
  num_input = Input(shape = (2,))
  
  ## Create embedding layer
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1)(cat_input)
  
  ## Flatten embedding layer
  embeding = Flatten()(embeding)
  
  ## Concatenate numerical layer and embedding layer
  combined = keras.layers.concatenate([num_input, embeding])
  
  ## Create normal dense layers
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  predictions = Dense(1, activation = 'linear')(dense2)
  
  ## Define final model
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  
  
  optimizer = tf.train.AdamOptimizer(learn) 
    
  model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
  
  
 
    
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=200)])
  
  
  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
  
  train_nn = sqrt(mse)
  
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
      
    
  train_nn_list.append(train_nn)
  test_nn_list.append(test_nn)
  
  ## Test on multiple sets
  for i in range(rounds):
    
    ## OLS
    df = get_data2(observations = observations, sigma = sigma, representation = "ols"  )

    train, test = split(df, test_size = test_size)
    
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    
    x = train.iloc[:,:-1].astype(float)
    y = train.iloc[:,-1]
    scalarX.fit(x)
    x = scalarX.transform(x)
    
    train = pd.concat([pd.DataFrame(x), y], axis = 1)
    train.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
    
    x1 = test.iloc[:,:-1].values
    y1 = test.iloc[:,-1].values
    
    x1 = scalarX.transform(x1)
    
    test = pd.concat([pd.DataFrame(x1), pd.DataFrame(y1)], axis = 1)
    test.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]

    
    results = smf.ols(formula = 'y ~ x1 + x2 + cat1 + cat2 +cat3 + cat4 + cat5 + cat6', data = sm.add_constant(train)).fit()
    
    train_pred_ols = results.fittedvalues
    train_ols = sqrt(metrics.mean_squared_error(train_pred_ols, train["y"]))
    
    test_pred_ols = results.predict(test)
    test_ols = sqrt(metrics.mean_squared_error(test_pred_ols, test["y"]))
    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    ## NN
    df = get_data2(observations = observations, sigma = sigma, representation = "nn"  )

    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
          
    ## Normalise  
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
   
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])

    
    
    
    
    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    train_nn = sqrt(mse)
    
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
        
      
    train_nn_list.append(train_nn)
    test_nn_list.append(test_nn)
  
  ## Find average MSE and standard deviation of MSE
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)                                  
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)                                  
                                    

  return(average_train_ols, average_test_ols, std_ols,average_train_nn, average_test_nn, std_nn)
    

In [8]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols,std_ols, train_nn, test_nn, std_nn = compare_regression_data2(observations = 1000, sigma = 0.1, test_size = 0.33, vector = [1024,512,64,1], learn = 0.01,output = 4,epochs = 1000, rounds = 30)

print("###########################  RMSE #######################")
print(" Training Error OLS: {}        |  Training Error NN: {} ".format(train_ols, train_nn))
print(" Testing Error OLS: {}         |  Testing Error NN: {}  ".format(test_ols, test_nn))
    
print("######################## Standard Deviation #############")
print(" Testing STD ols: {}           |  Testing std NN: {}   ".format(std_ols, std_nn))




Instructions for updating:
keep_dims is deprecated, use keepdims instead
###########################  RMSE #######################
 Training Error OLS: 0.10007895563492175        |  Training Error NN: 0.13816068617811625 
 Testing Error OLS: 0.09971303814054239         |  Testing Error NN: 0.31835462190324115  
######################## Standard Deviation #############
 Testing STD ols: 0.003704623113235151           |  Testing std NN: 0.010973979219339645   


## Regression - 2 continuous, one cat, non-linear

In [0]:
def get_data3(observations, sigma, representation):
 

  x1 = np.random.randint(0,100, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m feature
   
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  ## Add categorical variable (ranging between 1,7 to represent days of week)
  
  
  ## Create dataframe and create one-hot encoding
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  x = np.array(x)
  cats = np.array(cats)
  
  ## Set coefficients manually
  a = np.array([[1,1,1,1,7,7]])
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  w1 = 2
  w2 = 5
  
  ## Create non-linear relationship 
  y = np.dot(x1**2,w1) + np.dot(x2, w2) + np.dot(cats, a.T) + e
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  
 
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)


In [0]:
def compare_regression_data3(observations, sigma, test_size, vector, learn, output, epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  #NN
  df = get_data3(observations = observations, sigma = sigma, representation = "nn"  )

  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
        
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  cat_input = Input(shape=(1,)) 
  
  num_input = Input(shape = (2,))
  
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1)(cat_input)
  embeding = Flatten()(embeding)
  
  combined = keras.layers.concatenate([num_input, embeding])
  
  
  
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  predictions = Dense(1, activation = 'linear')(dense2)
  
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  
  
  optimizer = tf.train.AdamOptimizer(learn) 
    
  model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
  
  
  
    
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=200)])
  
  
  
  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
  
  train_nn = sqrt(mse)
  
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
      
    
  train_nn_list.append(train_nn)
  test_nn_list.append(test_nn)
  
  for i in range(rounds):
    
    ## OLS
    df = get_data3(observations = observations, sigma = sigma, representation = "ols"  )
    

    train, test = split(df, test_size = test_size)
    
    
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    
    x = train.iloc[:,:-1].astype(float)
    y = train.iloc[:,-1]
    scalarX.fit(x)
    x = scalarX.transform(x)
    
    train = pd.concat([pd.DataFrame(x), y], axis = 1)
    train.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
    
    x1 = test.iloc[:,:-1].values
    y1 = test.iloc[:,-1].values
    
    x1 = scalarX.transform(x1)
    
    test = pd.concat([pd.DataFrame(x1), pd.DataFrame(y1)], axis = 1)
    test.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
    
    results = smf.ols(formula = 'y ~ np.power(x1,2) + x2 + cat1 + cat2 +cat3 + cat4 + cat5 + cat6', data = sm.add_constant(train)).fit()
    
    train_pred_ols = results.fittedvalues
    train_ols = sqrt(metrics.mean_squared_error(train_pred_ols, train["y"]))
    
    test_pred_ols = results.predict(test)
    test_ols = sqrt(metrics.mean_squared_error(test_pred_ols, test["y"]))
    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    ##NN
    df = get_data3(observations = observations, sigma = sigma, representation = "nn"  )

    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
   
    
    
   
    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    train_nn = sqrt(mse)
    
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
        
      
    train_nn_list.append(train_nn)
    test_nn_list.append(test_nn)
  
  
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)                                  
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)                                  
                                    

  return(average_train_ols, average_test_ols, std_ols,average_train_nn, average_test_nn, std_nn)
    

In [11]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, std_nn, test_nn = compare_regression_data3(observations = 1000, sigma = 1, test_size = 0.33, vector = [1024,64,1], learn = 0.01,output = 4,epochs = 1000, rounds = 30)

print("###########################  RMSE #######################")
print(" Training Error OLS: {}        |  Training Error NN: {} ".format(train_ols, train_nn))
print(" Testing Error OLS: {}         |  Testing Error NN: {}  ".format(test_ols, test_nn))
    
print("######################## Standard Deviation #############")
print(" Testing STD ols: {}           |  Testing std NN: {}   ".format(std_ols, std_nn))




###########################  RMSE #######################
 Training Error OLS: 0.9900738923450195        |  Training Error NN: 1.194492879103494 
 Testing Error OLS: 1.012693731017992         |  Testing Error NN: 0.04134233040354216  
######################## Standard Deviation #############
 Testing STD ols: 0.0435276066798658           |  Testing std NN: 1.2304910000543108   


# Classification

## Classification, 2 continous

In [0]:
def get_cat_data1(observations,  sigma):

  ## Set coeeficients
  x1 = np.random.randint(0,100, size = [observations,1]).astype(float)/10
  x2 = np.random.randint(0,50, size = [observations,1]).astype(float)/10
  w1 = 5
  w2 = 2
  
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  
  x = np.hstack((x1,x2))
  y = np.dot(x1**2,w1) + np.dot(x2,w2)  + e
  
  ## Convert output to binary
  scaler = MinMaxScaler()
  y = scaler.fit_transform(y) ## Scale y value
  j = y    
  L = [0 if i < j.mean() else 1 for i in y]  ## Split by mean so that both categories have equal representation
  y = np.array(L).reshape(y.shape[0],1)
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df = pd.concat([df1, df2], axis = 1)
  
  df.columns = ["x1", "x2",  "y"]

  return(df)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)




In [0]:
def compare_classification_data1(observations, sigma, test_size,  vector, learn,  epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  def build_classification_model(vector, learn):
     model = Sequential()
     
     
     model.add(Dense(vector[0], activation = tf.nn.relu,
                     input_dim=X_train.shape[1]))
     
     for i in range(1,len(vector)-1):
       
        model.add(Dense(vector[i], activation=tf.nn.relu))
        
         
       
     model.add(Dense(1, activation = tf.nn.sigmoid))
     
   
     optimizer = tf.train.GradientDescentOptimizer(learn)
     
     ## Use binary crossentropy as loss function and accuracy as metric
     model.compile(optimizer=tf.train.AdamOptimizer(), 
                 loss= 'binary_crossentropy',
                 metrics=['accuracy'])
     return(model)
  
 
  df = get_cat_data1(observations = observations, sigma = sigma)
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
  
  ## Normalise data
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train)
  
  X_train = scalarX.transform(X_train)
  
  X_test = scalarX.transform(X_test)
  
  model = build_classification_model(vector, learn)
   
 
  model.fit(X_train,y_train, epochs=epochs,
                      validation_split=0.2, verbose=0)
  
  [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
  
  
  ## Round predictions to convert them to binary
  test_nn_pred = np.round(model.predict(X_test))
  
  test_nn = metrics.accuracy_score(test_nn_pred, y_test)
       
     
  train_nn_list.append(acc)
  test_nn_list.append(test_nn)
  
  
  for i in range(rounds):
    
    ## OLS
    df = get_cat_data1(observations = observations, sigma = sigma)

    train, test = split(df, test_size = test_size)

    
    ## Normalise
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    
    x = train.iloc[:,:-1].astype(float)
    y = train.iloc[:,-1]
    scalarX.fit(x)
    x = scalarX.transform(x)
    
    train = pd.concat([pd.DataFrame(x), y], axis = 1)
    train.columns = ["x1", "x2",  "y"]
    
    x1 = test.iloc[:,:-1].values
    y1 = test.iloc[:,-1].values
    
    x1 = scalarX.transform(x1)
    
    test = pd.concat([pd.DataFrame(x1), pd.DataFrame(y1)], axis = 1)
    test.columns = ["x1", "x2","y"]
    
    
    results = smf.logit(formula = 'y ~ np.power(x1,2) + x2 ', data = sm.add_constant(train)).fit(maxiter = 200, method = 'bfgs', disp = False)
    
    train_pred_ols = (results.fittedvalues > 0.5).astype(int)
        
        
    test_ols = results.predict(sm.add_constant(test))
    
    ## Round output to convert to binary
    test_ols = (test_ols > 0.5).astype(int).values.reshape(test_ols.size,1)
    
    
    train_ols = metrics.accuracy_score(train_pred_ols, train["y"])
    
    
    test_ols = metrics.accuracy_score(test_ols, test["y"])

    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    


    df = get_cat_data1(observations = observations, sigma = sigma)
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
    
    ## Normalise data
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train)
    
    X_train = scalarX.transform(X_train)
    
    X_test = scalarX.transform(X_test)
   
   
    
    [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
    
    
    ## Round ouput to convert it into binary
    test_nn_pred = np.round(model.predict(X_test))
    
    test_nn = metrics.accuracy_score(test_nn_pred, y_test)

        
      
    train_nn_list.append(acc)
    test_nn_list.append(test_nn)
  
  
  ## Return average accuracy and standard deviation of accuracy
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)  
  std_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols, std_ols, average_train_nn, average_test_nn, std_nn)
    

In [14]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, test_nn, std_nn = compare_classification_data1(observations = 100, sigma = 0.01, test_size = 0.33,  vector = [1024,64,1], learn = 0.01, epochs = 1000, rounds= 30)

print("########## Accuracy ##########")
print("Training Accuracy logistic: {}        |  Training Accuracy NN: {}".format(train_ols, train_nn))
print("Testing Accuracy logistic: {}         |  Testing Accuracy NN: {}".format(test_ols, test_nn))

print("########## Standard Deviation ###########")
print("Testing STD logistic: {}           |  Testing std NN: {}".format(std_ols, std_nn))



/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages

########## Accuracy ##########
Training Accuracy logistic: 1.0        |  Training Accuracy NN: 0.949927780538668
Testing Accuracy logistic: 0.993939393939394         |  Testing Accuracy NN: 0.9472140762463342
########## Standard Deviation ###########
Testing STD logistic: 0.012121212121212114           |  Testing std NN: 0.039222683573292054


/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


## **Classification, 2 continous, one categorical, linear **

In [0]:
def get_cat_data2(observations, sigma, representation):
 

  x1 = np.random.randint(0,100, size = [observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations,1])/10  ## Data points with n observations and m feature
   
  # Add categorical variable (ranging between 1,7 to represent days of week)  
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  
  
  
  ## Convert to dataframe and apply one-hot encoding
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  x = np.array(x)
  
  a = np.array([[2,5,1,1,1,1,7,7]])
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  
  ## Linear relationship
  y = np.dot(x,a.T) + e
  
  ## Convert output to binary, with equal number in each
  scaler = MinMaxScaler()
  y = scaler.fit_transform(y)
  j = y
    
  L = [0 if i < j.mean() else 1 for i in y]
    
  y = np.array(L).reshape(y.shape[0],1)
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)


In [0]:
def compare_classification_data2(observations, sigma, test_size,  vector, output, learn,  epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  df= get_cat_data2(observations = observations, sigma = sigma, representation = "nn")
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
   
  ## Normalise   
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  # Define the embedding input
  cat_input = Input(shape=(1,), dtype='float32') 
  
  ## Define numerical input
  num_input = Input(shape = (2,), dtype = 'float32')
  
  ## Define embedding layer
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1 )(cat_input)
  
  ## Flatten embedding layer
  embeding = Flatten()(embeding)
  
  ## Concatenate embedding with numerical
  combined = keras.layers.concatenate([num_input, embeding])
  
  
  ## Add dense layers
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  
  predictions = Dense(1, activation = 'sigmoid')(dense2)
  
  ## Define final model
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  
  model.compile(optimizer=tf.train.AdamOptimizer(), 
            loss= 'binary_crossentropy',
            metrics=['accuracy'])
  
  
  
  
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_acc', patience=150)])
  
  
   
  
  
  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
  
  ## Round output to convert to binary
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
       
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  train_nn_list.append(acc)
  test_nn_list.append(test_nn)
  
  for i in range(rounds):
    
    ##OLS
    df= get_cat_data2(observations = observations, sigma = sigma, representation = "nn")
    train, test = split(df, test_size = test_size)
    
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()

    x = train.iloc[:,:-1].astype(float)
    y = train.iloc[:,-1]
    scalarX.fit(x)
    x = scalarX.transform(x)
    
    train = pd.concat([pd.DataFrame(x), y], axis = 1)
    train.columns = ["x1", "x2", "cat", "y"]
    
    x1 = test.iloc[:,:-1].values
    y1 = test.iloc[:,-1].values
    
    x1 = scalarX.transform(x1)
    
    test = pd.concat([pd.DataFrame(x1), pd.DataFrame(y1)], axis = 1)
    test.columns = ["x1", "x2", "cat", "y"]
    
    results = smf.logit(formula = 'y ~ x1 + x2 + C(cat) ', data = sm.add_constant(train)).fit(method = 'bfgs', maxiter = 200, disp = False)
    
    train_pred_ols = (results.fittedvalues > 0.5).astype(int)
        
      
        
    test_ols = results.predict(sm.add_constant(test))
    test_ols = (test_ols > 0.5).astype(int).values.reshape(test_ols.size,1)
    
    
    train_ols = metrics.accuracy_score(train_pred_ols, train["y"])
    
    
    test_ols = metrics.accuracy_score(test_ols, test["y"])
        
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    
    
    df= get_cat_data2(observations = observations, sigma = sigma, representation = "nn")
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    

    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    ## Round output to convert to binary
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
    
         
    
    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    train_nn_list.append(acc)
    test_nn_list.append(test_nn)
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)
  
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols, std_ols, average_train_nn, average_test_nn, std_nn)
    

In [17]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, test_nn, std_nn = compare_classification_data2(observations = 200, sigma = 0.01, test_size = 0.33,  vector = [1024,64,1], output = 4, learn = 0.01,  epochs = 1000, rounds = 30)

print("########## Accuracy ##########")
print("Training Accuracy logistic: {}        |  Training Accuracy NN: {}".format(train_ols, train_nn))
print("Testing Accuracy logistic: {}         |  Testing Accuracy NN: {}".format(test_ols, test_nn))

print("########## Standard Deviation ###########")
print("Testing STD logistic: {}           |  Testing std NN: {}".format(std_ols, std_nn))



/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


########## Accuracy ##########
Training Accuracy logistic: 1.0        |  Training Accuracy NN: 0.9542609531832361
Testing Accuracy logistic: 0.9530303030303029         |  Testing Accuracy NN: 0.9565004887585532
########## Standard Deviation ###########
Testing STD logistic: 0.031137500286461325           |  Testing std NN: 0.03236895540527557


/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


## Classification, 2 continous, one categorical, non-linear

In [0]:
def get_cat_data3(observations, sigma,  representation):
 

  x1 = np.random.randint(0,100, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m feature
   
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  ## Add categorical variable (ranging between 1,7 to represent days of week)
  
  
  
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  x = np.array(x)
  cats = np.array(cats)
  
  a = np.array([[1,1,1,1,7,7]])
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  w1 = 2
  w2 = 5
  
  ## Create non-linear relationship
  y = np.dot(x1**2,w1) + np.dot(x2, w2) + np.dot(cats, a.T) + e
  
  scaler = MinMaxScaler()
  y = scaler.fit_transform(y)
  j = y
    
  L = [0 if i < j.mean() else 1 for i in y]
    
  y = np.array(L).reshape(y.shape[0],1)
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  
 
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)



In [0]:
def compare_classification_data3(observations, sigma, test_size,  vector, output, learn,  epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  df= get_cat_data3(observations = observations, sigma = sigma, representation = "nn")
    
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
  
  ## Normalise
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  # Define the embedding input
  cat_input = Input(shape=(1,), dtype='float32') 
  
  ## Define numerical input
  num_input = Input(shape = (2,), dtype = 'float32')
  
  ## Create embedding layer
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1 )(cat_input)
  embeding = Flatten()(embeding)
  
  combined = keras.layers.concatenate([num_input, embeding])
  
  ## Create normal layers
  
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  predictions = Dense(1, activation = 'sigmoid')(dense2)
  
  ## Define final model
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  model.compile(optimizer=tf.train.AdamOptimizer(), 
            loss= 'binary_crossentropy',
            metrics=['accuracy'])
  
  
  
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_acc', patience=150)])
  
  

  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
  
       
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  train_nn_list.append(acc)
  test_nn_list.append(test_nn)
  
  for i in range(rounds):
    
    ## OLS
    df = get_cat_data3(observations = observations, sigma = sigma,  representation = "nn")
    train, test = split(df, test_size = test_size)
    
   
    ## Normalise
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()

    x = train.iloc[:,:-1].astype(float)
    y = train.iloc[:,-1]
    scalarX.fit(x)
    x = scalarX.transform(x)
    
    train = pd.concat([pd.DataFrame(x), y], axis = 1)
    train.columns = ["x1", "x2", "cat", "y"]
    
    x1 = test.iloc[:,:-1].values
    y1 = test.iloc[:,-1].values
    
    x1 = scalarX.transform(x1)
    
    test = pd.concat([pd.DataFrame(x1), pd.DataFrame(y1)], axis = 1)
    test.columns = ["x1", "x2", "cat", "y"]
    
    results = smf.logit(formula = 'y ~ np.power(x1,2) + x2 + C(cat) ', data = sm.add_constant(train)).fit(method = 'bfgs', maxiter = 200,disp = False)
    
    train_pred_ols = (results.fittedvalues > 0.5).astype(int)
        
        
    test_ols = results.predict(sm.add_constant(test))
    test_ols = (test_ols > 0.5).astype(int).values.reshape(test_ols.size,1)
    
    
    train_ols = metrics.accuracy_score(train_pred_ols, train["y"])
    
    
    test_ols = metrics.accuracy_score(test_ols, test["y"])
            
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    
    
    df= get_cat_data3(observations = observations, sigma = sigma, representation = "nn")
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
    
    ## Normalise
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    

    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
      
      
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))

    
    
    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    train_nn_list.append(acc)
    test_nn_list.append(test_nn)
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)                        
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols,std_ols, average_train_nn, average_test_nn, std_nn)
    

In [20]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols,std_ols, train_nn, test_nn, std_nn = compare_classification_data3(observations = 1000, sigma = 0.1, test_size = 0.33,  vector = [1024,64,1], output = 4, learn = 0.01,  epochs = 1000, rounds = 30)

print("########## Accuracy ##########")
print("Training Accuracy logistic: {}        |  Training Accuracy NN: {}".format(train_ols, train_nn))
print("Testing Accuracy logistic: {}         |  Testing Accuracy NN: {}".format(test_ols, test_nn))

print("########## Standard Deviation ###########")
print("Testing STD logistic: {}           |  Testing std NN: {}".format(std_ols, std_nn))



/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


########## Accuracy ##########
Training Accuracy logistic: 1.0        |  Training Accuracy NN: 0.9892152140733996
Testing Accuracy logistic: 0.9936363636363635         |  Testing Accuracy NN: 0.9894428152492666
########## Standard Deviation ###########
Testing STD logistic: 0.00431042187744481           |  Testing std NN: 0.005649338524652927


/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


# No Embedding

## Regression

### Regression, data2

In [0]:
def NN_regression_2(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
    
  learn = 0.01
   
  
  
  
  df = get_data2(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
    
  ## Normalise  
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])

  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])

  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__regression_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "linear")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
      return model
    
    
    
    
  model = build__regression_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
 
  
  [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
  y_pred_nn = model.predict(X_test).astype(np.float64)
  
  
  train_nn = sqrt(mse)
  
  train.append(train_nn)
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
  test.append(test_nn)
  for i in range(rounds):
    
    
    
    df = get_data2(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    #
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
    
    ## Normalise
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])

    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
 
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
    
    
    [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
    y_pred_nn = model.predict(X_test).astype(np.float64)
    
    
    train_nn = sqrt(mse)
    
    train.append(train_nn)
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
    test.append(test_nn)
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)
  return(average_train, average_test, std)
  

In [22]:
## Repeat this cell varying number of observations and noise

train, test, std = NN_regression_2(observations = 1000, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

Train: 0.035601465395110166  |  Test: 0.038438390418405605     | std: 0.02652543424822429


### Regression, data3

In [0]:
def NN_regression_3(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
  
  learn = 0.01
   
  
  
  
  df = get_data3(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
  
  ## Normalise
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__regression_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "linear")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
      return model
    
    
    
    
  model = build__regression_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
   
  
  [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
  y_pred_nn = model.predict(X_test).astype(np.float64)
  
  
  train_nn = sqrt(mse)
  
  train.append(train_nn)
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
  test.append(test_nn)
  for i in range(rounds):
   
   
    
    
    df = get_data3(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
     
    # Normalise  
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
   
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
    
    
    [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
    y_pred_nn = model.predict(X_test).astype(np.float64)
    
    
    train_nn = sqrt(mse)
    
    train.append(train_nn)
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
    test.append(test_nn)
    
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)
  return(average_train, average_test, std)
  

In [24]:
## Repeat this cell varying number of observations and noise
train, test, std = NN_regression_3(observations = 1000, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

Train: 1.6326161998506696  |  Test: 1.6656533680421393     | std: 0.09666679457502793


## Classification

### Classification data2

In [0]:
def NN_classification_2(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
    
  learn = 0.01
   
  
  
  
  df = get_cat_data2(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
   
  ## Normalise  
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__classification_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "sigmoid")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='binary_crossentropy',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['accuracy'])
      return model
    
    
    
    
  model = build__classification_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
 
  
  [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
  test_nn_pred = np.round(model.predict(X_test).astype(np.float64))
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  
  
  
  
  train.append(acc)
  
  
  test.append(test_nn)
  for i in range(rounds):
    
    
    
    
    df = get_cat_data2(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
    
   
    
    [loss, acc] = model.evaluate(X_train,y_train, verbose=0)


    test_nn_pred = np.round(model.predict(X_test).astype(np.float64))

     

    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    
    
    
    
    train.append(acc)
    
    
    test.append(test_nn)
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)                        
  return(average_train, average_test, std)
  


In [26]:
## Repeat this cell varying number of observations and noise
train, test, std = NN_classification_2(observations = 200, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

Train: 0.9268175250759595  |  Test: 0.9340175953079181     | std: 0.03232464469948476


### Classification data3

In [0]:
def NN_classification_3(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
    
  learn = 0.01
   
  
  
  
  df = get_cat_data3(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  #
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
  
  ## Normalise
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])

  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])

  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__classification_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "sigmoid")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='binary_crossentropy',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['accuracy'])
      return model
    
    
    
    
  model = build__classification_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
 
  
  [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
  test_nn_pred = np.round(model.predict(X_test).astype(np.float64))
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  
  
  
  
  train.append(acc)
  
  
  test.append(test_nn)
    
  for i in range(rounds):
   
    
    
    df = get_cat_data3(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    #
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
  
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
   
    [loss, acc] = model.evaluate(X_train,y_train, verbose=0)


    test_nn_pred = np.round(model.predict(X_test).astype(np.float64))

     

    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    
    
    
    
    train.append(acc)
    
    
    test.append(test_nn)
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)                        
  return(average_train, average_test,std)
  


In [28]:
## Repeat this cell varying number of observations and noise
train, test, std = NN_classification_3(observations = 200, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

Train: 0.9359653344450515  |  Test: 0.926197458455523     | std: 0.03144051228949247
